In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# dataset load
data = pd.read_csv("datasets/diabetes2.csv")

# data prepare
X = data.drop('Outcome', axis='columns')
y = data['Outcome'] # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
data.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [40]:
class NeuralNetworkFromScratch:
    
    def __init__(self, input_layer_size, hidden_layer_size1, hidden_layer_size2, output_layer_size):
        self.weights1 = np.random.rand(input_layer_size, hidden_layer_size1)
        self.bias1 = np.zeros((1, hidden_layer_size1))
        self.weights2 = np.random.rand(hidden_layer_size1, hidden_layer_size2)
        self.bias2 = np.zeros((1, hidden_layer_size2))
        self.weights3 = np.random.rand(hidden_layer_size2, output_layer_size)
        self.bias3 = np.zeros((1, output_layer_size))
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def derivative_sigmoid(self, z):
        return z*(1-z)
        
    def forward(self, X):
        self.hidden1 = self.sigmoid(np.dot(X, self.weights1)+self.bias1)
        self.hidden2 = self.sigmoid(np.dot(self.hidden1, self.weights2)+self.bias2)
        self.output = self.sigmoid(np.dot(self.hidden2, self.weights3)+self.bias3)
        return self.output
        
    def backward(self, X, y, learning_rate):
        error = y-self.output
        d_output = error*self.derivative_sigmoid(self.output)
        
        error_hidden2 = d_output.dot(self.weights3.T)
        d_hidden2 = error_hidden2*self.derivative_sigmoid(self.hidden2)
        
        error_hidden1 = d_hidden2.dot(self.weights2.T)
        d_hidden1 = error_hidden1*self.derivative_sigmoid(self.hidden1)
        
        self.weights3 += self.hidden2.T.dot(d_output)*learning_rate
        self.bias3 += np.sum(d_output, axis=0, keepdims=True)*learning_rate
        
        self.weights2 += self.hidden1.T.dot(d_hidden2)*learning_rate
        self.bias2 += np.sum(d_hidden2, axis=0, keepdims=True)*learning_rate
        
        self.weights1 += X.T.dot(d_hidden1)*learning_rate
        self.bias1 = np.sum(d_hidden1, axis=0, keepdims=True)*learning_rate
        
        
    def fit(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, learning_rate)
            if epoch%1000==0:
                loss = np.mean(np.square(y-output))
                print(f"Loss: {loss}, epoch: {epoch}")
    
    def predict(self, X):
        return np.round(self.forward(X))
    

input_layer_size = X_train.shape[1]
hidden_layer_size1 = 5
hidden_layer_size2 = 4
output_layer_size = 1

model = NeuralNetworkFromScratch(input_layer_size, hidden_layer_size1, hidden_layer_size2, output_layer_size)

# model train
model.fit(X_train, y_train.to_numpy().reshape(-1,1),learning_rate=0.001, epochs=5000)

# model predict
y_pred = model.predict(X_test)

# evaluate
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)
matrix = confusion_matrix(y_test, y_pred)

print("Accuracy: ", acc)
print("Classification report: \n", report)
print("Confusion matrix: \n", matrix)

Loss: 0.4728315797537298, epoch: 0
Loss: 0.17027065212909026, epoch: 1000
Loss: 0.1576494480975937, epoch: 2000
Loss: 0.15389194886234545, epoch: 3000
Loss: 0.15077627094995966, epoch: 4000
Accuracy:  0.7916666666666666
Classification report: 
               precision    recall  f1-score   support

           0       0.82      0.89      0.85       130
           1       0.72      0.58      0.64        62

    accuracy                           0.79       192
   macro avg       0.77      0.74      0.75       192
weighted avg       0.79      0.79      0.79       192

Confusion matrix: 
 [[116  14]
 [ 26  36]]
